# Homework - Orbit

- Afet Ibadova - 453818

## Task:
Generate any time series containing trend and seasonality using Orbit - remember to combine these two components (https://orbit-ml.readthedocs.io/en/latest/tutorials/utilities_simulation.html). Then, conduct an experiment using DLT or LGT models (their different configurations) and check if you can predict the values of this time series. Carry out the full modeling process: divide the data into training and testing sets, train the models on the training set (perform backtesting analytics and hyperparameter tuning), and evaluate which model performs best on the testing set.

## Import libraries

In [1]:
!pip install orbit-ml pandas numpy

  Using cached orbit-ml-1.1.4.2.tar.gz (445 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached torch-2.1.2-cp39-cp39-win_amd64.whl (192.2 MB)
  Using cached arviz-0.17.0-py3-none-any.whl (1.7 MB)
  Using cached pyro_ppl-1.8.6-py3-none-any.whl (732 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached h5netcdf-1.3.0-py3-none-any.whl (43 kB)
  Using cached scipy-1.12.0-cp39-cp39-win_amd64.whl (46.2 MB)
  Using cached xarray_einstats-0.7.0-py3-none-any.whl (31 kB)
  Using cached setuptools-69.0.3-py3-none-any.whl (819 kB)
  Using cached pandas-2.2.0-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached matplotlib-3.8.2-cp39-cp39-win_amd64

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Afat\anaconda3\python.exe' 'C:\Users\Afat\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\Afat\AppData\Local\Temp\tmpggpnnvc0'
       cwd: C:\Users\Afat\AppData\Local\Temp\pip-install-zqej8ed6\orbit-ml_2932fd0d5ba241b6a511579c4692d08f
  Complete output (472 lines):
  running bdist_wheel
  running build
  running build_py
  Importing cmdstanpy...
  Installing cmdstan...
  CmdStan install directory: C:\Users\Afat\.cmdstan
  Installing CmdStan version: 2.31.0
  
  0.00B [00:00, ?B/s]
    0%|          | 0.00/47.3M [00:00<?, ?B/s]
    0%|          | 176k/47.3M [00:00<00:27, 1.80MB/s]
    1%|          | 464k/47.3M [00:00<00:35, 1.38MB/s]
    2%|1         | 784k/47.3M [00:00<00:24, 1.99MB/s]
    3%|2         | 1.25M/47.3M [00:00<00:16, 2.93MB/s]
    3%|3         | 1.61M/47.3M [00:00<00:15, 3.16MB/s]
    4%|4         | 1.98M/47.3M [00:00<00:14, 3.30MB/s]
    5%|4         | 2.3


Failed to build orbit-ml



   92%|#########1| 27296/29790 [00:32<00:02, 882.92it/s]


  100%|#########9| 29752/29790 [00:35<00:00, 921.60it/s]
  
  Unpacked download as cmdstan-2.31.0
  Building version cmdstan-2.31.0, may take several minutes, depending on your system.
  cmd: mingw32-make build -j8
  cwd: None
  23:36:58 - cmdstanpy - WARNING - CmdStan installation failed.
  Command "make build" failed
  Command: ['mingw32-make', 'build', '-j8']
  failed with error [WinError 2] The system cannot find the file specified
  
  CmdStan installation failed.
  Command "make build" failed
  Command: ['mingw32-make', 'build', '-j8']
  failed with error [WinError 2] The system cannot find the file specified
  
  Traceback (most recent call last):
    File "C:\Users\Afat\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 349, in <module>
      main()
    File "C:\Users\Afat\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py", line 331, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
    File "C:\Users\Afat\an

In [ ]:
import pandas as pd
import numpy as np
from orbit.models.dlt import DLTMAP, DLTFull
from orbit.models.lgt import LGTMAP, LGTFull
from orbit.utils.simulation import make_trend_arima_data
from orbit.diagnostics.backtest import backtest
from orbit.diagnostics.metrics import smape
from orbit.utils.simulation import make_trend, make_seasonality, make_regression
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from orbit.diagnostics.backtest import BackTester

## Generate TS with trend and seasonality

In [ ]:
# Set the parameters for the time series
n_periods = 365 * 5  # 5 years
trend_degree = 2  # quadratic trend
seasonality_period = 365  # yearly seasonality
seasonality_degree = 10  # complexity of seasonality
random_seed = 42  # for reproducibility

np.random.seed(random_seed)

# Generate components of the time series
trend = make_trend(n=n_periods, trend_degree=trend_degree, random_seed=random_seed)
seasonality = make_seasonality(n=n_periods, period=seasonality_period, degree=seasonality_degree, 
                               random_seed=random_seed)

# Combine trend and seasonality to create the time series
time_series = trend + seasonality

In [ ]:
# Create a DataFrame
dates = pd.date_range(start='2019-01-01', periods=n_periods, freq='D')
df = pd.DataFrame({'ds': dates, 'y': time_series})

In [ ]:
# Plot the generated time series
plt.figure(figsize=(12, 6))
plt.plot(df['ds'], df['y'], label='Generated Time Series')
plt.title('Synthetic Time Series with Trend and Seasonality')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# Returning the first few rows of the DataFrame
df.head()

##  Modelling

In [ ]:
# Splitting the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Showing the shape of training and testing sets
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)


In [ ]:
# Initialize DLT models
dlt_map = DLTMAP(response_col='y', date_col='ds')
dlt_full = DLTFull(response_col='y', date_col='ds')

# Initialize LGT models
lgt_map = LGTMAP(response_col='y', date_col='ds')
lgt_full = LGTFull(response_col='y', date_col='ds')

# Train the models on the training data
dlt_map.fit(train_df)
dlt_full.fit(train_df)
lgt_map.fit(train_df)
lgt_full.fit(train_df)


### Hyperparameter Tuning

In [ ]:
# Define a set of hyperparameters to test
param_grid = {
    'seasonality': [7, 14],
    'trend_type': ['linear', 'logistic'],
    'regression_penalty': ['lasso', 'ridge']
}

# Create a parameter grid
grid = ParameterGrid(param_grid)

# Initialize variables to store the best model and score
best_model = None
best_smape = float('inf')
best_params = None

# Iterate over the parameter grid
for params in grid:
    model = DLTMAP(response_col='response', **params)
    model.fit(train_data)
    pred = model.predict(test_data)
    score = smape(test_data['response'], pred['prediction'])

    if score < best_smape:
        best_smape = score
        best_model = model
        best_params = params

print("Best SMAPE:", best_smape)
print("Best Parameters:", best_params)


### Backtesting

In [ ]:


# Initialize backtester for the best model
backtester = BackTester(
    model=best_model,
    df=train_data,
    response_col='response',
    prediction_date_col='date',
    n_test=10  # number of test points
)

# Run backtesting
backtester.fit_predict()

# Extract backtest summary
backtest_summary = backtester.get_prediction_df()
print(backtest_summary.head())


In [ ]:
#best_model.fit(train_data)


In [ ]:
# Initialize the best model with the optimized hyperparameters
final_model = best_model_class(response_col='y', date_col='ds', **best_params)

# Train the model on the entire training dataset
final_model.fit(train_df)

# At this point, 'final_model' is trained with the best hyperparameters 
# and can be used for predictions or further analysis.


In [ ]:

# Predict on the test set
dlt_map_pred = dlt_map.predict(test_df)
dlt_full_pred = dlt_full.predict(test_df)
lgt_map_pred = lgt_map.predict(test_df)
lgt_full_pred = lgt_full.predict(test_df)

# Evaluate the models using a metric such as SMAPE (Symmetric Mean Absolute Percentage Error)
smape_dlt_map = smape(test_df['y'], dlt_map_pred['prediction'])
smape_dlt_full = smape(test_df['y'], dlt_full_pred['prediction'])
smape_lgt_map = smape(test_df['y'], lgt_map_pred['prediction'])
smape_lgt_full = smape(test_df['y'], lgt_full_pred['prediction'])

# Print the SMAPE scores
print("SMAPE - DLT MAP:", smape_dlt_map)
print("SMAPE - DLT Full:", smape_dlt_full)
print("SMAPE - LGT MAP:", smape_lgt_map)
print("SMAPE - LGT Full:", smape_lgt_full)


In [ ]:
# Predict on the test set for each model
dlt_map_pred = dlt_map.predict(test_df)
dlt_full_pred = dlt_full.predict(test_df)
lgt_map_pred = lgt_map.predict(test_df)
lgt_full_pred = lgt_full.predict(test_df)

# Calculate SMAPE for each model
smape_dlt_map = smape(test_df['y'], dlt_map_pred['prediction'])
smape_dlt_full = smape(test_df['y'], dlt_full_pred['prediction'])
smape_lgt_map = smape(test_df['y'], lgt_map_pred['prediction'])
smape_lgt_full = smape(test_df['y'], lgt_full_pred['prediction'])

# Print the SMAPE scores for comparison
print("SMAPE - DLT MAP:", smape_dlt_map)
print("SMAPE - DLT Full:", smape_dlt_full)
print("SMAPE - LGT MAP:", smape_lgt_map)
print("SMAPE - LGT Full:", smape_lgt_full)

# Determine the best model based on the lowest SMAPE score
best_model_name = min(
    {'DLT MAP': smape_dlt_map, 'DLT Full': smape_dlt_full, 'LGT MAP': smape_lgt_map, 'LGT Full': smape_lgt_full},
    key=lambda k: {'DLT MAP': smape_dlt_map, 'DLT Full': smape_dlt_full, 'LGT MAP': smape_lgt_map, 'LGT Full': smape_lgt_full}[k]
)

print(f"Best performing model: {best_model_name}")
